In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import and check the data

In [ ]:
# import the training data

train = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/train.csv')

train.head()

In [ ]:
# import the test data

test = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/test.csv')

test.head()

In [ ]:
# import sample submission

sample = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/sample_submission.csv')

sample.head()

In [ ]:
# look at the shape of the data

print('The shape of the train data is: {}'.format(train.shape))
print('The shape of the test data is: {}'.format(test.shape))

In [ ]:
# check for any missing data

print(train.isnull().sum())
print(test.isnull().sum())

In [ ]:
# check the Pearson correlation matrix for strongly correlated features

colormap = plt.cm.RdBu
plt.figure(figsize=(14,12))
plt.title('Pearson Correlation of Features', y=1.05, size=15)
sns.heatmap(train.iloc[:,1:].astype(float).corr(),linewidths=0.1,vmax=1.0, 
            square=True, cmap=colormap, linecolor='white', annot=True)



Since this is a regression problem, highly correlated features such as cont11 and cont12 should not be a problem and so will be left as they are.

# Defining the model

In [ ]:
# format the data appropriately

y = train['target']
X = train.iloc[:,1:15]
X_unknown = test.iloc[:,1:] # usually labeled as X_test

In [ ]:
# import cross validation

from sklearn.model_selection import cross_validate

# import the regressor

from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor


# define and fit the regressor

gbm = XGBRegressor(random_state=22,
                   n_estimators=3000,
                   learning_rate=0.05,
                   subsample=0.9,
                   colsample_bytree=0.9,
                   min_child_weight=3,
                   n_jobs=-1)

gbm.fit(X, y)

# cross validate

# scores = cross_validate(gbm, X, y, cv=3, scoring='neg_mean_squared_error', return_train_score=True)

# cross validation gave a RMSE score of [0.58354501 0.58461502 0.58498171]

In [ ]:
# predict the unknown dataset

y_test = gbm.predict(X_unknown)

# format the submission

sample['target'] = y_test

sample.head()

In [ ]:
# save to a csv

sample.to_csv("submission.csv", index=False)